In [151]:
import os
import pandas as pd
import numpy as np

In [152]:
os.getcwd()

'/Users/caseyh/Desktop/footy/data_wrangling'

In [172]:
path = '/Users/caseyh/Desktop/footy/stg_tables/'
teams_str = ['adelaide', 'brisbanel','carlton', 'collingwood','essendon', 'fremantle','geelong','goldcoast','gws',
        'hawthorn', 'melbourne', 'kangaroos','padelaide','richmond','stkilda','swans','westcoast','bullldogs']

In [173]:
stat_str = ['Kicks.csv',
 'Frees Against.csv',
 'Tackles.csv',
 'Bounces.csv',
 'Marks.csv',
 'Behinds.csv',
 'Hit Outs.csv',
 'Contested Marks.csv',
 'Disposals.csv',
 'Clearances.csv',
 'Inside 50s.csv',
 'Handballs.csv',
 'Contested Possessions.csv',
 'Clangers.csv',
 'Subs.csv',
 'Uncontested Possessions.csv',
 'Frees.csv',
 'Marks Inside 50.csv',
 'Goal Assists.csv',
 'Rebounds.csv',
 '% Played.csv',
 'Goals.csv',
 'One Percenters.csv']
cols = ['Player', 'round', 'team', 'opponent', 'Disposals','Kicks', 'Handballs','Goals','Behinds', 'Marks',
        'Clearances','Tackles','Contested Possessions','Uncontested Possessions',\
        'Hit Outs', 'Contested Marks','Marks Inside 50','Goal Assists',\
        'Frees','Frees Against',  \
         'Inside 50s','Rebounds', '% Played', 'Subs',\
       'Clangers','One Percenters', 'Bounces']
num_cols = ['Disposals','Kicks', 'Handballs','Frees Against', 'Tackles', 'Bounces',\
       'Marks', 'Behinds', 'Hit Outs', 'Contested Marks',\
       'Clearances', 'Inside 50s', 'Contested Possessions',\
       'Clangers', 'Uncontested Possessions', 'Frees',\
       'Marks Inside 50', 'Goal Assists', 'Rebounds', 'Goals',\
       'One Percenters']

In [174]:
# Data clean
# -, nan removal
# convert str to int
def clean(value):
    if value == '-':
        return 0
    elif type(value) == float:
        return np.nan
    elif type(int(value)) == int:
        return int(value)

In [175]:
# initial stat table 

for team in teams_str:
    table = pd.read_csv(f'{path}{team}/{stat_str[0]}').drop(columns = 'Tot')
    stat = stat_str[0].replace('.csv','')
    stats = table[(table['Player'] != 'Totals') & (table['Player'] != 'Opponent')]
    output = pd.melt(stats, id_vars=['Player']).rename(columns={'variable':'round','value':stat})
    
    # join rest of tables
    for s in stat_str[1:]:
        
        stat = s.replace('.csv','')
        table = pd.read_csv(f'{path}{team}/{s}').drop(columns = 'Tot')
        stats = table[(table['Player'] != 'Totals') & (table['Player'] != 'Opponent')]
        players_f = pd.melt(stats, id_vars=['Player']).rename(columns={'variable':'round','value':stat})
        output = output.merge(players_f, how = 'left', on = ['Player', 'round'])
    
    # get opponents
    opponent_f = table[table['Player'] == 'Opponent'].melt(id_vars = 'Player').drop(columns='Player').rename(columns={'variable':'round','value':'opponent'})
    
    output = output.merge(opponent_f, on= 'round')
    
    output['team'] = team
    
    output = output.fillna(np.nan)[cols]
    
    for col in num_cols:
        output[col] = output[col].apply(lambda x: clean(x))
    
    
    
    output.to_csv(f'/Users/caseyh/Desktop/footy/playerstats/{team}_player_stats.csv', index = False)
 

## All Player Stats Combined

In [176]:
lst = os.listdir('/Users/caseyh/Desktop/footy/playerstats/')
lst.remove('.DS_Store')
lst

['melbourne_player_stats.csv',
 'geelong_player_stats.csv',
 'kangaroos_player_stats.csv',
 'goldcoast_player_stats.csv',
 'bullldogs_player_stats.csv',
 'collingwood_player_stats.csv',
 'padelaide_player_stats.csv',
 'hawthorn_player_stats.csv',
 'richmond_player_stats.csv',
 'adelaide_player_stats.csv',
 'fremantle_player_stats.csv',
 'stkilda_player_stats.csv',
 'essendon_player_stats.csv',
 'westcoast_player_stats.csv',
 'brisbanel_player_stats.csv',
 'carlton_player_stats.csv',
 'gws_player_stats.csv',
 'swans_player_stats.csv']

In [177]:
output = pd.DataFrame({})

for i in lst:
    tmp = pd.read_csv(f'/Users/caseyh/Desktop/footy/playerstats/{i}')
    output = pd.concat([output,tmp])

In [178]:
opponent_mapping = {'RI':'RCH','GE':'GEE','NM':'NTH','PA':'POR','GC':'GCS','ME':'MEL','GW':'GWS','HW':'HAW','CA':'CAR','BL':'BRL','CW':'COL','AD':'ADE','FR':'FRE','WB':'WBD','SY':'SYD','WC':'WCE','SK':'STK','ES':'ESS'}
team_mapping = {'richmond':'RCH','geelong':'GEE','kangaroos':'NTH','padelaide':'POR','goldcoast':'GCS','melbourne':'MEL','gws':'GWS','hawthorn':'HAW','carlton':'CAR','brisbanel':'BRL','collingwood':'COL','adelaide':'ADE','fremantle':'FRE','bullldogs':'WBD','swans':'SYD','westcoast':'WCE','stkilda':'STK','essendon':'ESS'}

In [179]:
for key in opponent_mapping.keys():
    output.loc[output['opponent']==key, 'opponent'] = opponent_mapping[key]

In [180]:
for key in team_mapping.keys():
    output.loc[output['team']==key, 'team'] = team_mapping[key]

In [181]:
def rearrange(value):
    return ' '.join(' '.join(value.lower().split(', ')[::-1]).split(' '))

In [182]:
output['Player'] = output['Player'].apply(lambda x: rearrange(x))

In [183]:
output

,Player,round,team,opponent,Disposals,Kicks,Handballs,Goals,Behinds,Marks,...,Goal Assists,Frees,Frees Against,Inside 50s,Rebounds,% Played,Subs,Clangers,One Percenters,Bounces
0,jake bowey,R1,MEL,WBD,17.0,10.0,7.0,0.0,0.0,3.0,...,0.0,1.0,0.0,3.0,3.0,73.0,-,0.0,1.0,0.0
1,angus brayshaw,R1,MEL,WBD,22.0,14.0,8.0,0.0,0.0,10.0,...,0.0,1.0,1.0,2.0,4.0,86.0,-,3.0,0.0,0.0
2,ben brown,R1,MEL,WBD,15.0,12.0,3.0,4.0,1.0,7.0,...,1.0,1.0,0.0,2.0,1.0,81.0,-,2.0,1.0,0.0
3,kade chandler,R1,MEL,WBD,16.0,11.0,5.0,1.0,2.0,8.0,...,2.0,1.0,0.0,4.0,0.0,81.0,-,1.0,0.0,0.0
4,bayley fritsch,R1,MEL,WBD,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
463,angus sheldrick,R14,SYD,BRL,19.0,9.0,10.0,2.0,0.0,2.0,...,0.0,1.0,0.0,5.0,0.0,55.0,-,5.0,2.0,0.0
464,dylan stephens,R14,SYD,BRL,3.0,2.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,2.0,22.0,On,2.0,0.0,0.0
465,chad warner,R14,SYD,BRL,18.0,12.0,6.0,0.0,0.0,4.0,...,0.0,0.0,0.0,6.0,3.0,83.0,-,2.0,0.0,1.0
466,corey warner,R14,SYD,BRL,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [184]:
output.to_csv('/Users/caseyh/Desktop/footy/player_stats.csv', index = False)